In [1]:
import numpy as np
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import matplotlib.pyplot as plt
import librosa
import librosa.display
import sklearn
import sounddevice as sd
import soundfile as sf
from AimTTI_PLP import dev_TTI_PLP
from time import sleep
import csv  
from csv import DictWriter

def metal_to_number(metal):
    if metal=="Copper":
        return 2
    elif metal== "Brass":
        return 0
    elif metal== "Aluminium":
        return 1
    elif metal == "Steal":
        return 3
    else:
        print("incorrect spelling")
        return "ERROR"
    
def steel_to_number(metal):
    if metal=="Tool":
        return 2
    elif metal== "Stainless":
        return 0
    elif metal== "Bright_Black":
        return 1
    elif metal == "Galvanised":
        return 3
    elif metal == "Engineering":
        return 4
    else:
        print("incorrect spelling")
        return "ERROR"
def grade_to_number(metal):
    if metal== "St316":
        return 0
    elif metal== "St430":
        return 1
    elif metal=="St304":
        return 2
    elif metal == "En24":
        return 3
    elif metal == "BD2":
        return 4
    elif metal == "En3B":
        return 5
    elif metal == "En1A":
        return 6
    elif metal == "Black":
        return 7
    elif metal == "Galvanised":
        return 8
    elif metal == "Bright":
        return 9
    else:
        print("incorrect spelling")
        return "ERROR"

def defect_to_number(metal):
    if metal== "Control":
        return 0
    elif metal== "Scratch":
        return 1
    elif metal=="Hole":
        return 2
    elif metal == "Bends":
        return 3
    else:
        print("incorrect spelling")
        return "ERROR"
     
#GET TENSOR FOR SPECTROGRAM
def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = torchaudio.load(aud)
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

#VIEW SPECTROGRAM
def view_specrogram(aud,n_fft=1024, samplerate = 22050, hop_len=None):
    samples, sample_rate=librosa.load(aud, sr = samplerate)
    print(sample_rate)
    audio_stft = librosa.core.stft(samples, hop_length=hop_len, n_fft=n_fft)# gathering the absolute values for all values in our audio_stft 
    spectrogram = np.abs(audio_stft)# Converting the amplitude to decibel
    #librosa.feature.melspectrogram(S=spectrogram,sr=sample_rate, fmax=4000)
    #librosa.feature.spectral_bandwidth(*, y=None, sr=44100, S=None, n_fft=2048, hop_length=None, win_length=None, window='hann', center=True, pad_mode='constant', freq=None, centroid=None, norm=True, p=2)
    log_spectro = librosa.amplitude_to_db(spectrogram)# Plotting the short-time Fourier Transformation
    plt.figure(figsize=(20, 5))# Using librosa.display.specshow() to create our spectrogram
    librosa.display.specshow(log_spectro, sr=sample_rate, x_axis='time', y_axis='hz', cmap='magma')
    plt.colorbar(label='Decibels')
    #plt.title('Spectrogram (dB)', fontdict=dict(size=18))
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.show()
    
def record_and_bang(duration,fs,channels, port,wav_name, power = 12):
    TTI = dev_TTI_PLP(portstr=port)    
    idn=TTI.get_identity()
    print('GetIdentity (*IDN?) in AimTTI_PLP.py returns: {}'.format(idn))
    TTI._Vlimit = power
    TTI.SetVoltage = power
    TTI.SetCurrent = 3
    TTI.OutputOff
    sleep(0.1)
    myrec = sd.rec(int(duration*fs), samplerate=fs, channels = channels, blocking = False)
    for i in range(10): #it was 10
        sleep(0.6)
        TTI.OutputOn
        sleep(0.6)
        TTI.OutputOff
    TTI.close()
    sd.wait()
    sf.write(wav_name, myrec, fs)


from os import environ, pathsep
from platform import system

if not system() == 'Windows':
    raise RuntimeError('Thorlabs APT device drivers only supported under Windows OS.')

termpath = environ['PATH']
if not r'Thorlabs\APT\APT Server' in termpath:
    environ['PATH'] = r'C:\Program Files (x86)\Thorlabs\APT\APT Server' + pathsep + r'C:\Program Files\Thorlabs\APT\APT Server' + pathsep + termpath

try:
    from thorlabs_apt import core as apt_rs
except Exception as e:
    raise RuntimeError('Importing thorlabs_apt package failed (likely related to locating/loading APT.dll) with exception: ', e)

from time import sleep

class Rotation_Stage(object):
    '''
    Class to control a Thorlabs rotation stage using APT command library.
    Code is a simplified wrapper to thorlabs_apt package from PyPI.
    Defines the following user methods:
        rotate(speed=10.0) - starts constant angular velocity rotation (deg/sec) up to limit of motor

    Defines the following properties:
        identity - Get hardware information from motor controller. Returns tuple of strings containing hardware information.
        home - Send motor to home position. Returns True.
        angle - Property to Get or set motor position (in degrees, mod 360).
        stop - stop constant rotation. Returns True.
    '''
    def __init__(self, device_index=0):
        try:
            motor = apt_rs.list_available_devices()[device_index][1]
        except IndexError as e:
            raise RuntimeError('Cannot detect Thorlabs rotation stage controller. Check power and USB connection cable, then power-cycle the rotation stage controller to re-establish connection to PC.') from e
            
        self._motor = apt_rs.Motor(motor)
        self._acc, self._vmax =self._motor.get_velocity_parameter_limits()
        self._defaultv = 10.0
        self._defaultacc = 10.0
        self._motor.set_velocity_parameters(0.0, self._defaultacc, self._defaultv)

    @property
    def identity(self):
        '''
        Get hardware information from motor controller.
        Returns tuple of strings containing hardware information
        '''
        self._motor.identify()
        return self._motor.hardware_info

    @property
    def home(self):
        '''
        Send motor to home position.
        returns True
        '''
        self._motor.move_home(True)
        return True

    @property
    def angle(self):
        '''
        Property to Get or set motor position (in degrees, mod 360).
        '''
        return self._motor.position

    @angle.setter
    def angle(self, set_angle):
        self._motor.move_to(set_angle, True)

    def rotate(self, speed=10.0):
        '''
        Start rotation at constant angular velocity (degrees per second) set by speed (up to motor limit)
        returns True
        '''

        if (speed > self._vmax):
            speed = self._vmax
        #print('speed = {}').format(speed)
        self._motor.set_velocity_parameters(0.0, self._acc, float(speed))
        self._motor.move_velocity(2)
        return True

    @property
    def stop(self):
        '''
        Stop constant rotation.
        returns True
        '''
        self._motor.stop_profiled()
        sleep(1)
        self._motor.set_velocity_parameters(0.0, 10.0, 10.0)
        return True


In [3]:
#RUN THIS FOR STEEL
#AUTOMATION with varying power & angle
field_names = ['AUDIO','STEEL','Y', 'Y2', "DISTANCE", "ANGLE", "SHAPE", "AREA", "POWER", "THICK"]
channels=1
duration=13
port='COM9'
fs=int(44100)

##Changeables
distance=11         #mm away from metal to tip of microphone
shape="semicylinder"       
area= 55*20    #mm^2
thickness = 27
steel="Tool" #Tool, Bright_Black, Stainless, Engineering, Galvanised
grade="BD2" #St304, St316...
num=steel_to_number(steel) 
num2=grade_to_number(grade)
k=214


print(apt_rs.list_available_devices())
rs = Rotation_Stage()
print(rs.identity)
rs.home
print('homed')
#for loops
for i in range(5):
    rs.angle = i*45
    angle=i*45             #centre-of-mic angle to metal in degrees
    for j in range(9):
        wav_name='fold_2/{}_{}.wav'.format(steel,i*11+j+k) #change
        power = j*2+13.8
        record_and_bang(duration,fs,channels, port,wav_name, power)
        dict={'AUDIO':wav_name,'STEEL':steel,'Y':num, 'Y2':num2, "DISTANCE":distance, "ANGLE":angle, "SHAPE":shape, "AREA":area, "POWER":power, "THICK": thickness}
        with open('metadata_2/METADATA_2.csv', 'a', newline="") as f_object:

        #Pass the file object and a list of column names to DictWriter() You will get a object of DictWriter
            dictwriter_object = DictWriter(f_object, fieldnames=field_names)

        #Pass the dictionary as an argument to the Writerow()
            dictwriter_object.writerow(dict)

        #Close the file object
            f_object.close()

[(31, 83857315)]
(b'TDC001', b'SW Version 2.0.8', b'TDC001 DC Servo Drive')
homed
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?

FIBRE GLASS BELOW

In [3]:
#RUN THIS FOR FIBRE GLASS
#AUTOMATION with varying power & angle
field_names = ['AUDIO','DEFECT','Y', "DISTANCE", "ANGLE", "SHAPE", "AREA", "POWER", "THICK"]
channels=1
duration=13
port='COM9'
fs=int(44100)

##Changeables
distance=11         #mm away from metal to tip of microphone
shape="rectangle"       
area=62370*0.25            #mm^2
thickness = 0.6
defect="Bends"
num=defect_to_number(defect) #MAKE FUNTION DEFECT TO METAL
k = 0

print(apt_rs.list_available_devices())
rs = Rotation_Stage()
print(rs.identity)
rs.home
print('homed')
#for loops
for i in range(5):
    rs.angle = i*45
    angle=i*45             #centre-of-mic angle to metal in degrees
    for j in range(9):
        wav_name='fold_3/{}_{}.wav'.format(defect,i*11+j+k) 
        power = j*2+13.8
        record_and_bang(duration,fs,channels, port,wav_name, power)
        dict={'AUDIO':wav_name,'DEFECT':defect,'Y':num, "DISTANCE":distance, "ANGLE":angle, "SHAPE":shape, "AREA":area, "POWER":power, "THICK": thickness}
        with open('metadata_3/METADATA_3.csv', 'a', newline="") as f_object:

        #Pass the file object and a list of column names to DictWriter() You will get a object of DictWriter
            dictwriter_object = DictWriter(f_object, fieldnames=field_names)

        #Pass the dictionary as an argument to the Writerow()
            dictwriter_object.writerow(dict)

        #Close the file object
            f_object.close()

[(31, 83857315)]
(b'TDC001', b'SW Version 2.0.8', b'TDC001 DC Servo Drive')
homed
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?) in AimTTI_PLP.py returns: THURLBY THANDAR, PL303-P, 496139, 3.05-4.06
GetIdentity (*IDN?